In [2]:
!mamba install pandas

mambajs 0.19.13

Specs: xeus-python, numpy, matplotlib, pillow, ipywidgets>=8.1.6, ipyleaflet, scipy, pandas
Channels: emscripten-forge, conda-forge

Solving environment...
Solving took 2.893800000000745 seconds
  Name                          Version                       Build                         Channel                       
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
+ pandas                        3.0.0                         np22py313h9d9dc1e_0           emscripten-forge              
+ python-tzdata                 2025.3                        pyhd8ed1ab_0                  conda-forge                   
- pip                           25.3                          pyh145f28c_0                  conda-forge                   


In [3]:
!mamba install scikit-learn

mambajs 0.19.13

Specs: xeus-python, numpy, matplotlib, pillow, ipywidgets>=8.1.6, ipyleaflet, scipy, pandas, scikit-learn
Channels: emscripten-forge, conda-forge

Solving environment...
Solving took 1.680300000000745 seconds
  Name                          Version                       Build                         Channel                       
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
+ brotli-python                 1.2.0                         py313h33caa6c_0               emscripten-forge              
+ certifi                       2026.1.4                      pyhd8ed1ab_0                  conda-forge                   
+ charset-normalizer            3.4.4                         pyhd8ed1ab_0                  conda-forge                   
+ idna                          3.11                          pyhd8ed1ab_0                  conda-forge                   
+ joblib                        1.5.3 

In [4]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# K najblizih suseda

<h3>Učitavanje i analiza skupa podataka

In [5]:
data = pd.read_csv('data.csv')

In [6]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


<h3>Rad sa nedostajućim vrednostima

In [7]:
data.isna().any()

id                         False
diagnosis                  False
radius_mean                False
texture_mean               False
perimeter_mean             False
area_mean                  False
smoothness_mean            False
compactness_mean           False
concavity_mean             False
concave points_mean        False
symmetry_mean              False
fractal_dimension_mean     False
radius_se                  False
texture_se                 False
perimeter_se               False
area_se                    False
smoothness_se              False
compactness_se             False
concavity_se               False
concave points_se          False
symmetry_se                False
fractal_dimension_se       False
radius_worst               False
texture_worst              False
perimeter_worst            False
area_worst                 False
smoothness_worst           False
compactness_worst          False
concavity_worst            False
concave points_worst       False
symmetry_w

In [8]:
data['Unnamed: 32'].isna().all()

np.True_

In [9]:
data.drop('Unnamed: 32', axis=1, inplace=True)

In [10]:
data.drop('id', axis=1, inplace=True)

<h3>Podela na ulazne i ciljne atribute

Informacija o vrsti tumora se nalazi u koloni diagnosis (ciljni atribut), dok se ostali atributi koriste za predikciju (ulazni atributi).

In [11]:
Y = data['diagnosis']
X = data.drop('diagnosis', axis=1)

feature_names = X.columns

In [12]:
Y.unique()

<StringArray>
['M', 'B']
Length: 2, dtype: str

<h3>Provera balansiranosti klasa

In [13]:
M, B = 0, 0
for i in range(Y.size):
    if Y[i] == 'M':
        M+=1
    else:
        B+=1

print('M:', M, round(M/Y.size*100), '%')
print('B:', B, round(B/Y.size*100), '%')

M: 212 37 %
B: 357 63 %


Primećujemo blagu nebalansiranost (37%-63%).

Tipično, binarna klasifikacija sadrži jednu klasu koja predstavlja normalno stanje i drugu koja predstavlja abnormalno stanje.<br>
U našem primeru, benigni tumor predstavlja normalno stanje, a maligni abnormalno.<br> 
Klasi koja predstavlja normalno stanje se dodeljuje oznaka 0, dok se drugoj klasi dodeljuje oznaka 1.

In [14]:
Y = Y.replace({'M':1, 'B':0}).astype(int)
Y

0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: diagnosis, Length: 569, dtype: int32

<h3>Analiza ulaznih atributa

`Kod KNN algoritma` **`je potrebno uraditi normalizaciju podataka`**<br>
`KNN algoritam` **`je osetljiv na outlajere`**

**PRVO PODELA NA TRENING I TEST SKUP, PA ONDA NORMALIZACIJA I OUTLAJERI**

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, stratify=Y, random_state=41)

> <h4>Rad sa elementima van granica</h4><br>
    Pošto želimo da noramlizujemo podatke na raspon [0, 1], prvo je potrebno proveriti da li postoje elementi van granica

> Postoji veliki broj metoda za detekciju elemenata van granica:
> - Boxplot
> - Histogram
> - Mean and Standard Deviation
> - IQR (Inter Quartile Range)
> - Z-score
> - Percentile 

> U ovom zadatku ćemo iskoristiti IQR.

> 🧠 <u>Ideja metode</u><br>
> **IQR metoda** se zasniva na srednjih 50% podataka i zato je otporna na ekstremne vrednosti.
> Koristi
> - Q1 – 25. percentil
> - Q3 – 75. percentil
> - IQR = Q3 − 

>📐 <u>Granice za outlier-e</u><br>
>Element se smatra van granica ako važi:<br>
>$x < Q_1 - 1.5 \cdot IQR \quad \text{ili} \quad x > Q_3 + 1.5 \cdot IQR$

In [16]:
def IQR(X, feature_names):
    iqr = pd.DataFrame(0.0, index=feature_names,
                       columns=['lower', 'min', 'num_lower', 'upper', 'max', 'num_upper', 'percantage'])
    for name in feature_names:
        Q1, Q3 = X[name].quantile([0.25, 0.75])
        IQRv = Q3 - Q1
        upper = Q3 + 1.5 * IQRv
        lower = Q1 - 1.5 * IQRv

        num_lower = (X[name] < lower).sum()
        num_upper = (X[name] > upper).sum()
        percentage = round((num_lower + num_upper) / X.shape[0] * 100)

        iqr.loc[name, ['lower','upper','num_lower','num_upper','percantage','min','max']] = \
            [lower, upper, num_lower, num_upper, percentage, X[name].min(), X[name].max()]
    return iqr

In [17]:
iqr_table = IQR(X_train, feature_names)
iqr_table

,lower,min,num_lower,upper,max,num_upper,percantage
radius_mean,5.881250,6.981000,0.0,21.671250,28.11000,12.0,3.0
texture_mean,7.577500,10.720000,0.0,30.277500,39.28000,5.0,1.0
perimeter_mean,34.725000,43.790000,0.0,145.205000,188.50000,11.0,3.0
area_mean,-103.037500,143.500000,0.0,1310.462500,2501.00000,23.0,5.0
smoothness_mean,0.057775,0.052630,1.0,0.135055,0.16340,5.0,1.0
compactness_mean,-0.029744,0.019380,0.0,0.227926,0.34540,13.0,3.0
concavity_mean,-0.124894,0.000000,0.0,0.288056,0.42680,14.0,3.0
concave points_mean,-0.060858,0.000000,0.0,0.156303,0.20120,8.0,2.0
symmetry_mean,0.109413,0.106000,1.0,0.248912,0.30400,13.0,3.0
fractal_dimension_mean,0.044500,0.049960,0.0,0.079520,0.09744,11.0,3.0


> Dakle, možemo zaključiti da postoje autlajeri, ali ih neam mnogo i vrednosti ne odskaču previše.

> **Metode za rad sa autlajerima**<br>
> Ukoliko smatramo da će autlajeri previše uticati na model možemo primeniti više tehnika:<br>
> - Uklanjanje autlajera
> - Odsecanje - svi autlajeri se slikaju u granične vrednosti
> - Postavljanje na NaN + primena neke metode za rad sa nedostajućim vrednostima

`Domaci`: uraditi clipping

<h3>Normalizacija

Za normalizaciju ćemo iskoristiti `MinMaxScaler` 

Vrednosti min i max ćemo odrediti na osnovu trening skupa, a zatim primeniti transformaciju na test i trening skup.<br> 
<i>Ponovo naglašavamo, neophodno je da podelu na test i trening izvršimo pre normalizacije!

In [18]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
#scaler = MinMaxScaler(clip=True) #TRIK: ovde odradjen clipping!
scaler = MinMaxScaler()

scaler.fit(X_train) # 📌 OVO JE JAKO BITNO ❗ NE sme se fit na test skupu

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

<h3> KNN Algoritam

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
knn = KNeighborsClassifier()

In [22]:
knn.fit(X_train, Y_train) # treninranje modela

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",5
,"weights weights: {'uniform', 'distance'}, callable or None, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.Refer to the example entitled:ref:`sphx_glr_auto_examples_neighbors_plot_classification.py`showing the impact of the `weights` parameter on the decisionboundary.",'uniform'
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"p p: float, default=2Power parameter for the Minkowski metric. When p = 1, this is equivalentto using manhattan_distance (l1), and euclidean_distance (l2) for p = 2.For arbitrary p, minkowski_distance (l_p) is used. This parameter is expectedto be positive.",2
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'minkowski'
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.Doesn't affect :meth:`fit` method.",None


In [23]:
Y_pred = knn.predict(X_test)
Y_pred

/lib/python3.13/site-packages/threadpoolctl.py:1129: UserWarning: Unable to import LDSO from pyodide_js._module. This should never happen.
  warnings.warn(


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0])

<h4> Provera tacnosti

In [24]:
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(Y_test, Y_pred)
accuracy

0.972027972027972

<h4> Matrica konfuzije

$$
\begin{flalign*}
TN &\quad FP && \\
FN &\quad TP &&
\end{flalign*}
$$

- **TN** – True Negative  
- **FP** – False Positive  
- **FN** – False Negative  
- **TP** – True Positive 

In [25]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, Y_pred)

array([[89,  1],
       [ 3, 50]], dtype=int64)

<h4>F1 score

In [27]:
from sklearn.metrics import f1_score

In [29]:
F1 = f1_score(Y_test, Y_pred)
F1

0.9615384615384616

> <h4>Koliko suseda izabrati?

> Ako ne navedemo, podrazumevano je `n_neighbors = 5`
>
> Uvek koristi **NEPARAN K** (`za binarnu klasifikaciju`)